In [5]:
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

In [13]:
file = '/home/mango/Documents/unipd/magistrale/Earthquakes/SouthCalifornia-1982-2011_Physics-of-Data.dat' 
data = np.genfromtxt(file, dtype=None, delimiter=' ')         
header = ["id","id_prv","time","magnitude","x","y", "z"]
df = pd.DataFrame(data)
df.columns = header
df.shape

(110271, 7)

In [14]:
#plt.plot(df["time"], df["magnitude"])
#plt.plot(df["time"], df["x"])
#plt.plot(df["time"], df["y"])
#plt.plot(df["time"], df["z"])
#plt.plot(df["time"], df["id_prv"])


,id,id_prv,time,magnitude,x,y,z
0,0,-1,0.0,2.71,-2571956,-4627162,3520602


In [ ]:
def bins_histogram(data, precision=0.01, len_bin_min=0.1):
    """
    Params
    
    data: 1d numpy array
    precision: desired poisson precision of bins
    """
    N = int(pow(1/precision,2))
    data.sort()
    x = data[::N]
    y = data[1::N]
    if y.shape[0] == x.shape[0]:
        bins = np.concatenate((np.array(data.min()[np.newaxis]),(x[1:] + y[1:])/2, np.array(data.max()[np.newaxis])))
    else:
        bins = np.concatenate((np.array(data.min()[np.newaxis]),(x[1:-1] + y[1:])/2, np.array(data.max()[np.newaxis])))
    return bins

In [ ]:
data.shape

In [ ]:
data = np.array(df["time"])
data.sort()
diff_data = data[1:] - data[:-1]
print(diff_data.min())
bins = bins_histogram(diff_data, 0.01)
print(bins)
print(data)


In [ ]:


data1 = np.log(data)
bins1 = np.log(bins)

In [ ]:
data = np.array(df["time"])
data.sort()
diff_data = data[1:]-data[:-1]
plt.hist(diff_data[diff_data<200000], log=True)

In [ ]:
def select_bin_number(x, min_nbin = 7, fraction = 0.001):
    n_min = fraction*len(x)

    n, bin_extremes, _ = plt.hist(x, bins = min_nbin )
    plt.close()
    last_n = n[-1]

    if last_n > n_min:
        return min_nbin, bin_extremes
    else:
        i = min_nbin
        nbin = min_nbin
        while last_n < n_min and nbin < 100:
            nbin = nbin + 1
            #print('Attempt with nbin = {} \n'.format(nbin))
            n, _, _ = plt.hist(x, bins = nbin )
            plt.close()
            #print(n[i-1:])
            last_n = n[i-1:].sum()
            #print('Last n: ', last_n)
            #print('All n : ', n.sum())

        n, bin_extremes, _ = plt.hist(x, bins = nbin )
        plt.close()
        bins = np.concatenate((bin_extremes[:min_nbin],bin_extremes[-1:]))
        return nbin, bins 

In [ ]:
filtered =df[df["magnitude"]>3]
times = np.array(filtered["time"])
min_nbin = 7
fraction = 0.001
n_bin = fraction*len(diff)
n, bin_extremes, _ = plt.hist(x, bins = min_nbin)


In [ ]:
ev = {'1':{'2':{'6','7'},'3'}} 
for key in ev:
    for key2 in ev[key]:
        

In [ ]:
def evees(df, index=-1):
    dataset = {}
    df_filtered = df[df["id_prv"]==index]
    lenght =  df_filtered.shape[0]
    for i in range(lenght):
        event = df_filtered.iloc[i]
        dataset[str(event["id"])] = {"event" : event}
        dataset[str(event["id"])]["cilds"] = evees(df, index=int(event["id"]))
    return dataset



In [ ]:
depths = []
j = 0
def evees_light(df, index=-1, depth=0):
    global j
    j += 1
    dataset = {}
    df_filtered = df[df["id_prv"]==index]
    lenght =  df_filtered.shape[0]
    if lenght == 0:
        depths.append(depth)
    for i in range(lenght):
        event = df_filtered.iloc[i]
        #dataset[str(event["id"])] = {"event" : event}
        dataset[str(event["id"])] = {"depth" : depth}
        dataset[str(event["id"])]["cilds"] = evees_light(df, index=int(event["id"]), depth = depth + 1)
    return dataset